# Advent of Code day 9
## Part 1

In [72]:
import numpy as np

def unzip_disk_map(disk_map):
    result = []
    index = 0
    is_break = False
    for x in disk_map:
        if is_break:
            result.extend(["."] * int(x))
            is_break = False
        else:
            result.extend([str(index)] * int(x))
            index += 1
            is_break = True

    return result

In [73]:
def move_blocks(disk_map):
    first_dot = disk_map.index(".")
    last_non_dot = len(disk_map) - 1 - next((i for i, item in enumerate(reversed(disk_map)) if item != "."), len(disk_map) - 1) if "." in disk_map else -1

    while first_dot < last_non_dot:
        disk_map[first_dot], disk_map[last_non_dot] = disk_map[last_non_dot], disk_map[first_dot]

        first_dot = disk_map.index(".")
        last_non_dot = len(disk_map) - 1 - next((i for i, item in enumerate(reversed(disk_map)) if item != "."), len(disk_map) - 1) if "." in disk_map else -1


    return disk_map

In [74]:
def checksum(disk_map):
    id = 0
    runsum = 0
    for x in disk_map:
        if x == ".":
            id += 1
            continue
        else:
            runsum += id * int(x)
            id += 1
    return runsum

In [75]:
with open("day9.txt", 'r') as file:
    # Read just the first line
    disk_map = file.readline()
unzipped_disk_map = unzip_disk_map(disk_map)
moved_blocks_disk_map = move_blocks(unzipped_disk_map)
print(checksum(moved_blocks_disk_map))

6288707484810


## part 2

In [76]:
import itertools
def print_disk_map_list(disk_map):
    s = ""
    for x in disk_map:
        s += str(x)
    print(s)

def find_group_indices_itertools(data_list, target_value):
    # This approach works best if you need to find *all* groups, not just the first or last
    index = 0
    for key, group in itertools.groupby(data_list):
        group_length = len(list(group))
        if str(key) == str(target_value):
            # Calculate the start and end points for this specific group
            start = index
            end = index + group_length - 1
            return (start, end)
            # If you only need the first one, return (start, end) here
        index += group_length
    return (start, end)

def find_leftmost_group(data_list, target_value, min_size_n, max_index=None):
    """
    Finds the start and end indices of the leftmost consecutive group 
    of target_value that has a length of at least min_size_n.
    Only considers indices less than max_index.
    """
    if max_index is None:
        max_index = len(data_list)
    
    current_start = None
    count = 0

    for i, value in enumerate(data_list):
        if i >= max_index:
            break
            
        if str(value) == str(target_value):
            if count == 0:
                # Mark the start of a new potential group
                current_start = i
            count += 1
        else:
            # The group of target_value ended. Check its size.
            if count >= min_size_n:
                # Found the leftmost qualifying group!
                return (current_start, i - 1)
            
            # Reset count and start index for the next potential group
            count = 0
            current_start = None
            
    # Check if the list ended with a qualifying group
    if count >= min_size_n:
        return (current_start, len(data_list) - 1)

    # If no qualifying group was found
    return (None, None)

def move_blocks_2(disk_map):
   
    largest_id = max([s for s in set(disk_map)])
    
    for id in range(int(largest_id),-1,-1):
        #print_disk_map_list(disk_map)
        #get the indices of id
        id_indices = find_group_indices_itertools(disk_map, id)
        size = id_indices[1] - id_indices[0] + 1

        #find the leftmost place that can fit the id
        switch_indicies = find_leftmost_group(disk_map, ".", size, max_index=id_indices[0])

        if switch_indicies == (None, None): #if there is no room, continue
            continue
        else: #swap the locations
            disk_map[switch_indicies[0]:switch_indicies[0]+ size], disk_map[id_indices[0]:id_indices[1]+1] = disk_map[id_indices[0]:id_indices[1]+1], disk_map[switch_indicies[0]:switch_indicies[0]+ size]
    return disk_map

In [77]:
with open("day9.txt", 'r') as file:
    # Read just the first line
    disk_map = file.readline()
unzipped_disk_map = unzip_disk_map(disk_map)
moved_blocks_disk_map = move_blocks_2(unzipped_disk_map)
print(checksum(moved_blocks_disk_map))

6311837662089
